In [28]:
def plotY(y_train):
    counts=pd.Series(y_train).value_counts()
    unique_labels=pd.Series(y_train).value_counts().index
    plt.bar(unique_labels,counts)
    print(pd.Series(y_train).value_counts())

def plotResults(model,X_test_reshaped):
    prediction = model.predict(X_test_reshaped)
#     myprediction=[np.argmax(pred) for pred in prediction ]
    myprediction = np.argmax(prediction, axis=1)
    counts=pd.Series(myprediction).value_counts()
    unique_labels=pd.Series(myprediction).value_counts().index
    plt.bar(unique_labels,counts)
    print(pd.Series(myprediction).value_counts())
    return myprediction

def reshape_dataset(X, n_steps):
    n_features = X.shape[1]
    X_reshaped = []
    for i in range(len(X)):
        if i<(len(X) - n_steps + 1):
            X_reshaped.append(X[i:i+n_steps,:])
        else:
            s=n_steps-(len(X)-i)
            aux=np.append(X[i:len(X),:],X[0:s,:])
            aux=aux.reshape(n_steps,n_features)
            X_reshaped.append(aux)
    return np.array(X_reshaped)

import keras.backend as K

def f1_score(y_true, y_pred):
    """Computes the F1 score."""
    y_true = K.cast(y_true, "float32")
    y_pred = K.round(y_pred)
    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1_score = 2 * precision * recall / (precision + recall + K.epsilon())
    return f1_score

def finalPreprocessing(df,df_test,order):
    Y_train=df.loc[:,'Class']
    X_train=df.iloc[:,1:-1]

    #Fill NA's
    X_train_interpolate=pd.DataFrame()
    X_train_interpolate = X_train.interpolate(method='spline', order=order)
    X_train_interpolate=X_train_interpolate.iloc[1:,:]
    Y_train=Y_train.iloc[1:]
    
    #Sampling 
    counts=pd.Series(Y_train).value_counts()
    sampling_strategy = {0: int(counts[0]*0.7)}
    rus = RandomUnderSampler(random_state=0,sampling_strategy=sampling_strategy)
    X_train_interpolate_resampled, y_train_interpolate_resampled = rus.fit_resample(X_train_interpolate, Y_train)

    counts=pd.Series(y_train_interpolate_resampled).value_counts()
    sampling_strategy = {0:  int(counts[0]), 1: int(counts[0]*0.6), 2:int(counts[0]*0.6), 3: int(counts[0]*0.6), 4: int(counts[0]*0.6)}
    ros = RandomOverSampler(random_state=0,sampling_strategy=sampling_strategy)
    X_train_interpolate_resampled, y_train_interpolate_resampled = ros.fit_resample(X_train_interpolate_resampled, y_train_interpolate_resampled)

    #Test 
    X_test=df_test.iloc[:,1:]
    X_test_interpolate = X_test.interpolate(method='spline', order=3)
    X_test_interpolate=X_test_interpolate.iloc[2:,:]

    #Get values
    X_train=X_train_interpolate_resampled.values
    X_test=X_test_interpolate.values
    y_train=y_train_interpolate_resampled.values

    #normalize
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

    #Reshape
    X_train_reshaped = reshape_dataset(X_train, n_steps=10)
    X_test_reshaped = reshape_dataset(X_test, n_steps=10)

    return X_train_reshaped,y_train,X_test_reshaped
  


def testingPreprocessing(df,order):
    Y_train=df.loc[:,'Class']
    X_train=df.iloc[:,1:-1]

    #Fill NA's
    X_train_interpolate=pd.DataFrame()
    #if method=='interpolation':
    X_train_interpolate = X_train.interpolate(method='spline', order=order)
    X_train_interpolate=X_train_interpolate.iloc[1:,:]
    Y_train=Y_train.iloc[1:]

    #Sampling 
    counts=pd.Series(Y_train).value_counts()
    sampling_strategy = {0: int(counts[0]*0.7)}
    rus = RandomUnderSampler(random_state=0,sampling_strategy=sampling_strategy)
    X_train_interpolate_resampled, y_train_interpolate_resampled = rus.fit_resample(X_train_interpolate, Y_train)

    counts=pd.Series(y_train_interpolate_resampled).value_counts()
    sampling_strategy = {0:  int(counts[0]), 1: int(counts[0]), 2:int(counts[0]), 3: int(counts[0]), 4: int(counts[0])}
    ros = RandomOverSampler(random_state=0,sampling_strategy=sampling_strategy)
    X_train_interpolate_resampled, y_train_interpolate_resampled = ros.fit_resample(X_train_interpolate_resampled, y_train_interpolate_resampled)

    #Test 
    X_train,X_test,y_train,y_test= train_test_split(X_train_interpolate_resampled, y_train_interpolate_resampled, test_size=0.3, random_state=42)

    #Get values
    X_train=X_train.values
    X_test=X_test.values
    y_train=y_train.values
    y_test=y_test.values

    #normalize
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

    #Reshape
    X_train_reshaped = reshape_dataset(X_train, n_steps=10)
    X_test_reshaped = reshape_dataset(X_test, n_steps=10)

    return X_train_reshaped,X_test_reshaped,y_train,y_test


def RealTestingPreprocessing(df,order):
    Y_train=df.loc[:,'Class']
    X_train=df.iloc[:,1:-1]
    
    #train test split
    X_train, X_test, Y_train, y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=42)
    X_train=X_train.sort_index()
    Y_train=Y_train.sort_index()
    X_test=X_test.sort_index()
    y_test=y_test.sort_index()


    #Fill NA's
    X_train_interpolate=pd.DataFrame()
    X_train_interpolate = X_train.interpolate(method='spline', order=order)
    X_train_interpolate=X_train_interpolate.iloc[1:,:]
    Y_train=Y_train.iloc[1:]

#     #Sampling 
#     counts=pd.Series(Y_train).value_counts()
#     sampling_strategy = {0: int(counts[0]*0.7)}
#     rus = RandomUnderSampler(random_state=0,sampling_strategy=sampling_strategy)
#     X_train_interpolate_resampled, y_train_interpolate_resampled = rus.fit_resample(X_train_interpolate, Y_train)

    counts=pd.Series(Y_train).value_counts()
    sampling_strategy = {0:  int(counts[0]), 1: int(counts[1]*2), 2:int(counts[2]*1.5), 3: int(counts[3]*2), 4: int(counts[0]*1.5)}
    ros = RandomOverSampler(random_state=0,sampling_strategy=sampling_strategy)
    X_train_interpolate_resampled, y_train_interpolate_resampled = ros.fit_resample(X_train_interpolate, Y_train)


    #Test 
    X_test_interpolate = X_test.interpolate(method='spline', order=3)
    X_test_interpolate=X_test_interpolate.iloc[2:,:]
    y_test=y_test[2:]

    #Get values
    X_train=X_train_interpolate_resampled.values
    X_test=X_test_interpolate.values
    y_train=y_train_interpolate_resampled.values
    y_test=y_test.values

    #normalize
    mean = X_train.mean(axis=0)
    std = X_train.std(axis=0)
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

    #Reshape
    X_train_reshaped = reshape_dataset(X_train, n_steps=10)
    X_test_reshaped = reshape_dataset(X_test, n_steps=10)

    return X_train_reshaped,y_train,X_test_reshaped,y_test
  

def interpolate(X_train,Y_train,method,order):
    if method=='linear':
        X_train_linear=pd.DataFrame()
        X_train_linear = X_train.interpolate(method='linear')
        X_train_linear=X_train_linear.iloc[1:,:]
        Y_train=Y_train.iloc[1:]
        return X_train_linear,Y_train
    if method=='spline':
        X_train_spline=pd.DataFrame()
        X_train_spline = X_train.interpolate(method='spline', order=order)
        X_train_spline=X_train_spline.iloc[1:,:]
        Y_train=Y_train.iloc[1:]
        return X_train_spline,Y_train
    elif method=='KNN':
        imputer = KNNImputer(n_neighbors=11)
        data_imputed = imputer.fit_transform(X_train)
        return data_imputed
    if method=='RF':
        X_train_RF=pd.DataFrame()
        rf_imputer = RandomForestImputer(random_state=42)
        X_train_RF= rf_imputer.fit_transform(X_train)
        return X_train_RF
    if method=='XGB':
        X_train_XGB=pd.DataFrame()
        xgb_imputer = xgb.XGBImputer()
        X_train_XGB= xgb_imputer.fit_transform(X_train)
        return X_train_XGB
    
def zeros(row):
    n=np.where(row.isna())[0]
    if len(n)>0:
        if sum(row[n[-1]+1:n[-1]+11])==0:
            row[n]=0
    return row
def mode(solutions):
    all_preds={}
    #initialize
    for i in range(len(solutions[0])):
        all_preds[i]=[]
    #store solutions
    for sol in solutions:
        for position, value in enumerate(sol):
            all_preds[position].append(value)
    #get mode
    mode={}
    for i in range(len(solutions[0])):
        preds=all_preds[i]
        numbers={0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
        for j in preds:
            numbers[j]+=1
        mode[i]=max(numbers, key=numbers.get)
            
    return mode

In [27]:
df = pd.read_csv('Train_set.csv')
def reshape_dataset(X, n_steps):
    n_features = int(X.shape[1]/n_steps)
    X_reshaped = []
    for i in range(len(X)):
        if i<(len(X) - n_steps + 1):
            X_reshaped.append(X[i:i+n_steps,:])
        else:
            s=n_steps-(len(X)-i)
            aux=np.append(X[i:len(X),:],X[0:s,:])
            aux=aux.reshape(n_steps,n_features)
            X_reshaped.append(aux)
    return np.array(X_reshaped)